In [73]:
# Cricket subset extraction
import requests
sentence_list=[]
def get_conceptnet_data(concept):

    for i in concept:
        base_url = "https://api.conceptnet.io"
        endpoint = f"/c/en/{i}?offset={0}&limit={10000}"
        url = base_url + endpoint

        response = requests.get(url)
        data = response.json()
        edges = data.get('edges', "")
        for edge in edges:
            surface_text = edge.get('surfaceText', '')
            if None==surface_text:
                pass
            else:
                sentence_list.append(surface_text)


    return sentence_list

concept = ["ground","stadium","coach","partner"]
# concept=["cricket"]
conceptnet_data = get_conceptnet_data(concept)
print(conceptnet_data)



['[[floor]] is related to [[ground]]', '[[ground]] is related to [[floor]]', '[[ground]] is related to [[earth]]', '[[ground]] is related to [[dirt]]', 'You are likely to find [[oil]] in [[the ground]]', 'You are likely to find [[dirt ]] in [[the ground]].', '[[ground]] is related to [[under]]', '*Something you find on [[the ground]] is [[rocks]]', '[[ground]] is related to [[below]]', '[[ground]] is related to [[feet]]', '[[mole]] is related to [[ground]]', '*Something you find on [[the ground]] is [[grass]]', '[[land]] is related to [[ground]]', '[[grass]] is related to [[ground]]', '[[ground]] is related to [[soil]]', '[[sand]] is related to [[ground]]', '[[ground]] is the opposite of [[sky]]', '[[ground]] is related to [[surface]]', '*Something you find on [[the ground]] is [[snow]]', 'You are likely to find [[a fungus]] in [[ground]]', '*Something you find on [[the ground]] is [[penny]]', '*Something you find on [[the ground]] is [[a pebble]]', '*Something you find on [[the ground

In [74]:
print(len(sentence_list))

690


In [41]:
# Anika, Moinul and Nafis Cricket subset
import pandas as pd
df_main = pd.read_csv('Cricket_sentences.csv')
sentences = df_main['English sentences'].tolist()

# Printing the converted list.
print(sentences)

['Players are entertainers.', 'Stumping is a method of getting out.', 'Bowlers are players', 'Umpires are the decision takers.', 'Test cricket is the longest form of cricket.', 'Bails falling by the hit of the batsman costs a wicket.', "If the nonstriker is out of line and the bowler hits the nonstriker's stumps that causes a wicket", 'Getting injured in the game may affect the career.', 'Sledging the opponent may bring some benefits for the team.', 'Ball tampering may change the outcome of a match.', 'Stumps are found on both sides of the pitch.', 'Two umpires are seen on the field.', 'Audience stays in the gallery.', 'Security is seen at the entrance of the stadium.', 'Players rest in the dressing room.', 'Umpires conduct the match.', 'Umpires ensure the rule of cricket.', 'Audience cheers the players.', 'Captain leads the team.', 'Scoreboard shows the situation of the match.', 'To win a match, players need to be focused.', "To get an lbw wicket, the ball must hit the batsman's pad f

In [ ]:
pip install rouge


In [ ]:
#Fine grained and semantic similarity
import tensorflow as tf
import tensorflow_hub as hub


embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
similar={}
dissimilar={}
close={}
similarity=[]
for i in range(len(sentence_list)):
    for j in range(len(sentences)):

        embeddings = embed([sentence_list[i], sentences[j]])
        similarity_score = tf.reduce_sum(tf.multiply(embeddings[0], embeddings[1])).numpy()
        similarity.append(similarity_score)
        if similarity_score>=0.5:
            similar[similarity_score]=[sentence_list[i], sentences[j]]
        if similarity_score<0.5 and similarity_score>=0.1:
            close[similarity_score]=[sentence_list[i], sentences[j]]
        if similarity_score<0.1:
            dissimilar[similarity_score]=[sentence_list[i], sentences[j]]



print(f"Semantic Similarity: {similarity}")

In [ ]:
# print(len(similarity))
f1=open("similar.txt","w")
for i in similar.keys():
    f1.write(str(i)+" ")
    for j in similar[i]:
        f1.write(j+","+" ")
    f1.write("\n")
f1.close()

In [ ]:
f1=open("dissimilar.txt","w")
for i in dissimilar.keys():
    f1.write(str(i)+" ")
    for j in dissimilar[i]:
        f1.write(j+","+" ")
    f1.write("\n")
f1.close()

In [ ]:
f1=open("close.txt","w")
for i in close.keys():
    f1.write(str(i)+" ")
    for j in close[i]:
        f1.write(j+","+" ")
    f1.write("\n")
f1.close()

In [75]:
#1,2 and 3 using lemmatization
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

sub_d={}
obj_d={}
obj_correct={}
sub_correct={}
for sentence in sentences:
    for i in sentence_list:

        doc = nlp(sentence)

        subject = None
        predicate = None
        obj = None

        for token in doc:
            if "subj" in token.dep_:
                subject = token.text
            if "obj" in token.dep_:
                obj = token.text
            if "ROOT" in token.dep_:
                predicate = token.text
        if subject==None or obj==None or predicate==None:
            pass
        else:
            new_sentence=i.replace("[", "")
            new_sentence=new_sentence.replace("]", "")
            subject=subject.lower()
            obj=obj.lower()
            predicate=predicate.lower()
            new_subject=lemmatizer.lemmatize(subject)
            new_obj=lemmatizer.lemmatize(obj)
            new_predicate=lemmatizer.lemmatize(predicate)
            if new_subject in new_sentence and new_predicate in new_sentence:
                doc2 = nlp(new_sentence)

                subject = None
                predicate = None
                obj = None

                for token in doc2:
                    if "subj" in token.dep_:
                        subject = token.text
                    if "obj" in token.dep_:
                        obj = token.text
                    if "ROOT" in token.dep_:
                        predicate = token.text
                if obj!=None:
                    obj_d[obj]=[sentence,new_sentence]
                    if obj in new_obj or new_obj in obj:
                        obj_correct[obj]=[sentence,new_sentence]
            if new_obj in new_sentence and new_predicate in new_sentence:
                doc2 = nlp(new_sentence)

                subject = None
                predicate = None
                obj = None

                for token in doc2:
                    if "subj" in token.dep_:
                        subject = token.text
                    if "obj" in token.dep_:
                        obj = token.text
                    if "ROOT" in token.dep_:
                        predicate = token.text
                if subject!=None:
                    sub_d[subject]=[sentence,new_sentence]
                    if subject in new_subject or new_subject in subject:
                        sub_correct[subject]=[sentence,new_sentence]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [76]:
#Rouge test
from rouge import Rouge


for key, value in sub_correct.items():
    reference, candidate = value[0], value[1]


    rouge = Rouge()


    scores = rouge.get_scores(candidate, reference)


    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']


    sub_correct[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [77]:
#Rouge test
from rouge import Rouge


for key, value in obj_correct.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()


    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']


    obj_correct[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [78]:
#Rouge test
from rouge import Rouge


for key, value in sub_d.items():
    reference, candidate = value[0], value[1]


    rouge = Rouge()


    scores = rouge.get_scores(candidate, reference)


    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    sub_d[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [79]:
#Rouge test
from rouge import Rouge


for key, value in obj_d.items():
    reference, candidate = value[0], value[1]


    rouge = Rouge()


    scores = rouge.get_scores(candidate, reference)


    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']


    obj_d[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [80]:
f1=open("lemmatization correct subject.txt","w")
for i in sub_correct.keys():
    f1.write(str(i)+":"+" ")
    for j in sub_correct[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [81]:
f1=open("lemmatization subject.txt","w")
for i in sub_d.keys():
    f1.write(str(i)+":"+" ")
    for j in sub_d[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [82]:
f1=open("lemmatization correct object.txt","w")
for i in obj_correct.keys():
    f1.write(str(i)+":"+" ")
    for j in obj_correct[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [83]:
f1=open("lemmatization object.txt","w")
for i in obj_d.keys():
    f1.write(str(i)+":"+" ")
    for j in obj_d[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [84]:
#4 and 5 using lemmatization
import spacy


nlp = spacy.load("en_core_web_sm")

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

s={}
o={}
relation = ["is a", "related to", "form of", "has a", "used for", "capable of", "purpose of", "location for", "causes", "subevent", "in order for", "as a property", "prevented by", "wants", "creates", "synonym", "antonym", "distinct", "derived from", "symbol of", "defined as", "way to", "manner of", "located near", "in context of", "made of", "symmetric", "common origin", "can be done to"]
for sentence in sentences:
    for i in sentence_list:
        doc = nlp(sentence)

        subject = None
        predicate = None
        obj = None

        for token in doc:
            if "subj" in token.dep_:
                subject = token.text
            if "obj" in token.dep_:
                obj = token.text
            if "ROOT" in token.dep_:
                predicate = token.text
        if subject!=None:

            new_sentence=i.replace("[", "")
            new_sentence=new_sentence.replace("]", "")
            subject=subject.lower()

            new_subject=lemmatizer.lemmatize(subject)
            if new_subject in new_sentence:
                for j in relation:
                    if j in new_sentence:
                        doc2 = nlp(new_sentence)

                        subject = None
                        predicate = None
                        obj = None

                        for token in doc2:
                            if "subj" in token.dep_:
                                subject = token.text
                            if "obj" in token.dep_:
                                obj = token.text
                            if "ROOT" in token.dep_:
                                predicate = token.text
                        if obj!=None:
                            o[obj]=[sentence,new_sentence]
        if obj!=None:
            new_sentence=i.replace("[", "")
            new_sentence=new_sentence.replace("]", "")

            obj=obj.lower()
            new_obj=lemmatizer.lemmatize(obj)
            if new_obj in new_sentence:
                for j in relation:
                    if j in new_sentence:
                        doc2 = nlp(new_sentence)

                        subject = None
                        predicate = None
                        obj = None

                        for token in doc2:
                            if "subj" in token.dep_:
                                subject = token.text
                            if "obj" in token.dep_:
                                obj = token.text
                            if "ROOT" in token.dep_:
                                predicate = token.text
                        if subject!=None:
                            s[subject]=[sentence,new_sentence]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [87]:
#Rouge test
from rouge import Rouge


for key, value in s.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()

    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    s[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [86]:
#Rouge test
from rouge import Rouge

for key, value in o.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()

    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    o[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [88]:
f1=open("lemm_object.txt","w")
for i in o.keys():
    f1.write(str(i)+":"+" ")
    for j in o[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [89]:
f1=open("lemm_subject.txt","w")
for i in s.keys():
    f1.write(str(i)+":"+" ")
    for j in s[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [ ]:
print(s)
print(o)

In [ ]:
print(sub_d)
print(obj_d)
print(obj_correct)
print(sub_correct)

In [90]:
#1,2 and 3 using stemming
import spacy

nlp = spacy.load("en_core_web_sm")

from nltk.stem import PorterStemmer


porter_stemmer = PorterStemmer()

sub_d={}
obj_d={}
obj_correct={}
sub_correct={}
for sentence in sentences:
    for i in sentence_list:

        doc = nlp(sentence)

        subject = None
        predicate = None
        obj = None

        for token in doc:
            if "subj" in token.dep_:
                subject = token.text
            if "obj" in token.dep_:
                obj = token.text
            if "ROOT" in token.dep_:
                predicate = token.text
        if subject==None or obj==None or predicate==None:
            pass
        else:
            new_sentence=i.replace("[", "")
            new_sentence=new_sentence.replace("]", "")
            subject=subject.lower()
            obj=obj.lower()
            predicate=predicate.lower()
            new_subject=porter_stemmer.stem(subject)
            new_obj=porter_stemmer.stem(obj)
            new_predicate=porter_stemmer.stem(predicate)
            if new_subject in new_sentence and new_predicate in new_sentence:
                doc2 = nlp(new_sentence)

                subject = None
                predicate = None
                obj = None

                for token in doc2:
                    if "subj" in token.dep_:
                        subject = token.text
                    if "obj" in token.dep_:
                        obj = token.text
                    if "ROOT" in token.dep_:
                        predicate = token.text
                if obj!=None:
                    obj_d[obj]=[sentence,new_sentence]
                    if obj in new_obj or new_obj in obj:
                        obj_correct[obj]=[sentence,new_sentence]
            if new_obj in new_sentence and new_predicate in new_sentence:
                doc2 = nlp(new_sentence)

                subject = None
                predicate = None
                obj = None

                for token in doc2:
                    if "subj" in token.dep_:
                        subject = token.text
                    if "obj" in token.dep_:
                        obj = token.text
                    if "ROOT" in token.dep_:
                        predicate = token.text
                if subject!=None:
                    sub_d[subject]=[sentence,new_sentence]
                    if subject in new_subject or new_subject in subject:
                        sub_correct[subject]=[sentence,new_sentence]

In [91]:
#Rouge test
from rouge import Rouge


for key, value in sub_correct.items():
    reference, candidate = value[0], value[1]


    rouge = Rouge()


    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    sub_correct[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [92]:
#Rouge test
from rouge import Rouge

for key, value in obj_correct.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()

    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    obj_correct[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [93]:
#Rouge test
from rouge import Rouge

for key, value in sub_d.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()

    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    sub_d[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [94]:
#Rouge test
from rouge import Rouge

for key, value in obj_d.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()

    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']


    obj_d[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [95]:
f1=open("stemmed correct subject.txt","w")
for i in sub_correct.keys():
    f1.write(str(i)+":"+" ")
    for j in sub_correct[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [96]:
f1=open("stemmed correct object.txt","w")
for i in obj_correct.keys():
    f1.write(str(i)+":"+" ")
    for j in obj_correct[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [97]:
f1=open("stemmed subject.txt","w")
for i in sub_d.keys():
    f1.write(str(i)+":"+" ")
    for j in sub_d[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [98]:
f1=open("stemmed object.txt","w")
for i in obj_d.keys():
    f1.write(str(i)+":"+" ")
    for j in obj_d[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [ ]:
print(sub_d)
print(obj_d)
print(obj_correct)
print(sub_correct)

In [99]:
#4 and 5 using stemming
import spacy

nlp = spacy.load("en_core_web_sm")

from nltk.stem import PorterStemmer

porter_stemmer = PorterStemmer()

s={}
o={}
relation = ["is a", "related to", "form of", "has a", "used for", "capable of", "purpose of", "location for", "causes", "subevent", "in order for", "as a property", "prevented by", "wants", "creates", "synonym", "antonym", "distinct", "derived from", "symbol of", "defined as", "way to", "manner of", "located near", "in context of", "made of", "symmetric", "common origin", "can be done to"]
for sentence in sentences:
    for i in sentence_list:
        doc = nlp(sentence)

        subject = None
        predicate = None
        obj = None

        for token in doc:
            if "subj" in token.dep_:
                subject = token.text
            if "obj" in token.dep_:
                obj = token.text
            if "ROOT" in token.dep_:
                predicate = token.text
        if subject!=None:

            new_sentence=i.replace("[", "")
            new_sentence=new_sentence.replace("]", "")
            subject=subject.lower()

            new_subject=porter_stemmer.stem(subject)
            if new_subject in new_sentence:
                for j in relation:
                    if j in new_sentence:
                        doc2 = nlp(new_sentence)

                        subject = None
                        predicate = None
                        obj = None

                        for token in doc2:
                            if "subj" in token.dep_:
                                subject = token.text
                            if "obj" in token.dep_:
                                obj = token.text
                            if "ROOT" in token.dep_:
                                predicate = token.text
                        if obj!=None:
                            o[obj]=[sentence,new_sentence]
        if obj!=None:
            new_sentence=i.replace("[", "")
            new_sentence=new_sentence.replace("]", "")

            obj=obj.lower()
            new_obj=porter_stemmer.stem(obj)
            if new_obj in new_sentence:
                for j in relation:
                    if j in new_sentence:
                        doc2 = nlp(new_sentence)

                        subject = None
                        predicate = None
                        obj = None

                        for token in doc2:
                            if "subj" in token.dep_:
                                subject = token.text
                            if "obj" in token.dep_:
                                obj = token.text
                            if "ROOT" in token.dep_:
                                predicate = token.text
                        if subject!=None:
                            s[subject]=[sentence,new_sentence]

In [100]:
#Rouge test
from rouge import Rouge

for key, value in s.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()

    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    s[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [101]:
#Rouge test
from rouge import Rouge

for key, value in o.items():
    reference, candidate = value[0], value[1]

    rouge = Rouge()

    scores = rouge.get_scores(candidate, reference)

    rouge_1_score = scores[0]['rouge-1']['f']
    rouge_2_score = scores[0]['rouge-2']['f']
    rouge_l_score = scores[0]['rouge-l']['f']

    o[key] += [rouge_1_score, rouge_2_score, rouge_l_score]



In [102]:
f1=open("stemm_subject.txt","w")
for i in s.keys():
    f1.write(str(i)+":"+" ")
    for j in s[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [103]:
f1=open("stemm_object.txt","w")
for i in o.keys():
    f1.write(str(i)+":"+" ")
    for j in o[i]:
        f1.write(str(j)+","+" ")
    f1.write("\n")
f1.close()

In [ ]:
print(s)
print(o)